# Imports

In [ ]:

import sys
sys.path.append('/../')
import numpy as np
import matplotlib.pyplot as plt
import traveltime as tt
import pinv
import copy
from scipy.io import loadmat

%matplotlib inline


# for reloading submodules without restarting kernel
%load_ext autoreload
%autoreload 2

# ignore plot warnings in gstools
import warnings
warnings.filterwarnings('ignore')


# Introduction

This notebook gives some hints in terms of how to approach the assignement and get to grips with the PINV package. 

Please note the PINV package is a not a finished package, it is still under development by T. Meier-Hansen so some variables and functions in the package are not finished or some parts are hard-coded in the source code. However, the package contains all the functions and variable settings needed to solve the assignment. It is highly recommended that you walk though the tutorial notebooks first, they are very much relevant, and you may well get stuck if you do not look at these first.

Additionally, it can be a good idea to look in the "sippi.pdf" file in literature folder, as it provides explanations of similar variables in the MATLAB version of the package and the naming convention has been kept the same. So a strategy for understanding a function or variable below could be (in prioritized order):
- Search for the function/variable name or slight variations in "sippi.pdf", "Hansen_etal_2013_SIPPI1.pdf" and "Hansen_etal_2013_SIPPI2.pdf". Read! 
- Experiment with a small number of iterations and see which effect different variables have
- Look into source code (search function in VScode is very helpful here), but keep in mind that this requires reading the code - it does not have a lot of comments. Specifically line 107-132 in ```extended_metropolis.py``` provides a bit of insight into what order things are called in.

Make sure you can run everything below and make some test diagnostic plots before you start to run a large number of iterations (this can take a long time!). Note that you can load the hdf5 files after the run has completed, so if you run the model and later want to look at results you can load them from the hdf5, instead of running everything again. 

# Load data

In [ ]:

M=loadmat('data/BHRS_2D.mat')
data_obs = M['traveltimes'].squeeze()
data_std = M['traveltimes_std'].squeeze()
ndata, ndim = M['sources'].shape
sources = M['sources'].squeeze()
receivers = M['receivers'].squeeze()

dx = .25
dy = dx
min_x = np.min(M['sources'][:, 0])
max_x = np.max(M['receivers'][:, 0])
min_y = np.min(np.hstack((M['sources'][:, 1],  M['receivers'][:, 1])))
max_y = np.max(np.hstack((M['sources'][:, 1],  M['receivers'][:, 1])))


# Setup prior

The prior is set up by assigning various fields to the prior object. First we assign a name and the type of prior we want to use, let us us the Cholesky method which is based on a cholesky decomposition of the covariance model.

In [ ]:
im = 0
prior = [{}]
prior[im]['name'] = 'Cholesky, velocity (m/ns)'
prior[im]['method'] = 'cholesky'

prior[im]['x'] = np.arange(min_x - 2 * dx, max_x + 2 * dx, dx)
prior[im]['y'] = np.arange(min_y - 2 * dy, max_y + 2 * dy, dy)
domain_shape = (len(prior[im]['y']), len(prior[im]['x']))  # for reshaping later

# Set properties of prior: !dummy values, you need to find the correct values based on the training images!
mean0=0.1
h=8.75
var=1e-05

prior[im]['m0'] = mean0
prior[im]['Cm'] = '{} Sph({})'.format(var, h)


# Setup forward

In [ ]:

##%% SETUP FORWARD
forward = {}
forward['S'] = M['sources'][:, 0:2]
forward['R'] = M['receivers'][:, 0:2]
forward['function'] = 'forward_traveltime'


# Setup data

In [ ]:

##%% SETUP DATA
data=[{}]
data[0]['d_obs']=data_obs
data[0]['d_std']=data_std


# Test setup

In [ ]:

##%% TEST SETUP
m = pinv.prior.simulate(prior) # realisations of velocities from prior distribution
d = pinv.forward.forward(m,forward,prior)
logL = pinv.likelihood(d,data)
print('LogL(g(m)) = %6.3f' % (logL[0]) )


## Plot model and rays

In [ ]:

##%% PLOT MODEL and RAYS
plt.figure(figsize=(7, 7))
plt.pcolor(prior[im]['x'], prior[im]['y'], m[0])
plt.gca().set_aspect('equal')
plt.gca().xaxis.set_label_position('top') 
for i in range(ndata):
    plt.plot([sources[i,0], receivers[i,0]], [sources[i,1], receivers[i,1]], 'w-', linewidth=0.2)
plt.gca().invert_yaxis()
plt.xlabel('X (m)', fontsize=15, labelpad=10)
plt.ylabel('Z (m)', fontsize=15, labelpad=10)
cbar = plt.colorbar()
cbar.set_label('Velocity (m/ns)', fontsize=15, labelpad=20)
plt.show()


## Plot data predictions from single forward pass

In [ ]:

plt.figure(figsize=(9, 5))
plt.plot(d[0], label='Data prediction from one forward pass')
plt.plot(data_obs, label='Observed data')
plt.ylabel('Traveltimes (ns)')
plt.xlabel('Datum #')
plt.legend()
plt.grid('grey')
plt.show()


# Do Metropolis sampling

In [ ]:

##%% SETUP METROPOLIS
mcmc={}
mcmc["n_ite"]=5000  # Too short - just for illustration!
#mcmc["n_ite"]=25000 # Longer but still too short?
mcmc["n_sample"]=100  # number of realizations to store
mcmc["i_plot"]=100  # plotting option, not working
mcmc['i_progress']=250 # how often progress should be updated

# Update step
mcmc['i_update_step']=50; # how often step length is adjusted
mcmc['i_update_step_max']=int(mcmc["n_ite"]/5); # for how long will the step length be updated
prior[0]['step_min']=0.001; # minimum step length


In [ ]:

pinv.verbose_level=2  # verbosity level, does not affect model, only print updates

mcmc_out = pinv.sampling.metropolis_chains(copy.deepcopy(forward), copy.deepcopy(prior), copy.deepcopy(data), mcmc)

# Might report error saving stricts using "deepdish" (which copies deep python structures), this is not a problem for this assignment

# Plot results

## Plot log-likelihood

In [ ]:

pinv.plot.plot_likelihood(mcmc_out['hdf5filename'])


## Plot acceptance rate

In [ ]:

pinv.plot.plot_acceptance_rate(mcmc_out['hdf5filename'])


## Plot acceptance rate histogram

In [ ]:

# The chain is not saved in mcmc variable but stored in hdf5 file
import h5py
f = h5py.File(mcmc_out['hdf5filename'], 'r')

# Acc rate is computed as an avg acc rate for a window of maximum 200 indices on each side of the current bin (bins of size 10, "n_ite" = 1000 gives 100 bins)
wi = 100
di = 10
v = 'C%d/i_acc' % 0
i_acc=f[v][im,:]

N=len(i_acc)
i_range=np.arange(0, N, di)
accrate = np.zeros(len(i_range))
k=0
for i in i_range:
    i1 = np.max([0, i - wi])
    i2 = np.min([N, i + wi])
    accrate[k] = np.sum(i_acc[i1:i2])/(i2-i1)
    k=k+1
    
fig = plt.figure(figsize=(9, 5))
plt.hist(accrate)
plt.show()


## Plot random posterior samples

In [ ]:

# Get posterior sample from saved outputs
m_all = pinv.sampling.get_sample_from_hdf5(mcmc_out['hdf5filename'], im=0, ic=0, nskip=0)

# Plot sequence of output models from posterior sample
random_index = np.random.randint(0, 100, 9)
m_plot = m_all[random_index]

fig = plt.figure(figsize=(10,10))
for i, posterior_sample in enumerate(m_plot):
    ax = fig.add_subplot(3, 3, i + 1) # this line adds sub-axes
    ax.imshow(posterior_sample.reshape(domain_shape))
plt.show()
